# DABstep Benchmark Qwen2.5-Coder-32B Baseline

This notebook will guide you though submitting a Qwen2.5-Coder-32B baseline to the DABstep leaderboard.

* Live 🤗 Leaderboard: https://huggingface.co/spaces/adyen/DABstep
* Benchmark 🤗 Dataset: https://huggingface.co/datasets/adyen/DABstep
* LLM Agent Framework by 🤗: https://github.com/huggingface/smolagents/tree/main



## Environment Setup

We need to setup:
* **HuggingFace Token:** In order to make free API calls to HuggingFace Inference API you must have a HF account, the API verifies this by checking your account's token. This token will not be used for anything else.
* **Benchmark context files:** In order to solve the benchmark tasks the agent will need to reference documentation and analyze data which is spread out across multiple files, just like a real Data Analyst would.

### HuggingFace Token Setup

In [113]:
import time
import os
import json
import re
import datasets
import pandas as pd
from smolagents import CodeAgent
from smolagents.agents import ActionStep
from smolagents.models import OpenAIServerModel
from huggingface_hub import hf_hub_download
from datetime import datetime

# Load OpenRouter API key from secrets
openrouter_key = None
openrouter_path = os.path.abspath(os.path.join(os.getcwd(), "..", "secrets", "openrouter_credentials.txt"))
if os.path.exists(openrouter_path):
    with open(openrouter_path, "r") as f:
        content = f.read()
    # Extract API key - look for pattern like sk-or-v1-...
    key_match = re.search(r"sk-or-v1-[A-Za-z0-9]+", content)
    if key_match:
        openrouter_key = key_match.group(0)
        print("✓ Loaded OpenRouter API key from secrets")
    else:
        print("⚠️ Warning: No valid OpenRouter key found in", openrouter_path)
else:
    print("⚠️ Error: OpenRouter credentials file not found at", openrouter_path)
    raise FileNotFoundError("OpenRouter credentials required to run this notebook")

# Try to import DABstep utilities
try:
    from dabstep_benchmark.utils import evaluate
    DABSTEP_AVAILABLE = True
except ImportError:
    print("Warning: DABstep benchmark utilities not available.")
    DABSTEP_AVAILABLE = False
    def evaluate(*args, **kwargs):
        raise RuntimeError("DABstep utilities not installed.")

notebook_start_time = time.time()
print(f'Right now is {datetime.now().isoformat()}')


✓ Loaded OpenRouter API key from secrets
Right now is 2025-11-11T17:22:22.905850


#### Download context files
First we download the context files from the [Benchmark's Dataset](https://huggingface.co/datasets/adyen/DABstep) so that our agent can access them.


In [114]:
CONTEXT_FILENAMES = [
    "data/context/acquirer_countries.csv",
    "data/context/payments-readme.md",
    "data/context/payments.csv",
    "data/context/merchant_category_codes.csv",
    "data/context/fees.json",
    "data/context/merchant_data.json",
    "data/context/manual.md",
]
# Store data locally in the repo, not in /tmp
DATA_DIR = os.path.join(os.path.dirname(os.path.abspath(".")), "data", "context_files")
os.makedirs(DATA_DIR, exist_ok=True)

for filename in CONTEXT_FILENAMES:
    hf_hub_download(
        repo_id="adyen/DABstep",
        repo_type="dataset",
        filename=filename,
        local_dir=DATA_DIR,
        force_download=False
    )

CONTEXT_FILENAMES = [f"{DATA_DIR}/{filename}" for filename in CONTEXT_FILENAMES]

for file in CONTEXT_FILENAMES:
    if os.path.exists(file):
        print(f"{file} exists.")
    else:
        print(f"{file} does not exist.")

/home/mykola/repos/dabstep_test/data/context_files/data/context/acquirer_countries.csv exists.
/home/mykola/repos/dabstep_test/data/context_files/data/context/payments-readme.md exists.
/home/mykola/repos/dabstep_test/data/context_files/data/context/payments.csv exists.
/home/mykola/repos/dabstep_test/data/context_files/data/context/merchant_category_codes.csv exists.
/home/mykola/repos/dabstep_test/data/context_files/data/context/fees.json exists.
/home/mykola/repos/dabstep_test/data/context_files/data/context/merchant_data.json exists.
/home/mykola/repos/dabstep_test/data/context_files/data/context/manual.md exists.


## Agent

Here we will setup a simple zero-shot prompt for the agent. It has two parts, the general prompt and then a quick outline of which files are available.

In [115]:
# Available OpenRouter models
MODELS_OPENROUTER = {
    "qwen2.5": "qwen/qwen-2.5-coder-32b-instruct",
    "qwen-coder-480b": "qwen/qwen3-coder",
    "deepseek-v3.1": "deepseek/deepseek-chat-v3.1",
    "deepseek-v3-terminus": "deepseek/deepseek-v3.1-terminus",
    "deepseek-v3.2": "deepseek/deepseek-v3.2-exp",
    "kimi-k2-0905": "moonshotai/kimi-k2-0905",
    "kimi-k2-thinking": "moonshotai/kimi-k2-thinking",
    "glm-4.5-air": "z-ai/glm-4.5-air",
    "glm-4.5": "z-ai/glm-4.5",
    "glm-4.6": "z-ai/glm-4.6",
    "gpt-oss-20b": "openai/gpt-oss-20b",
    "gpt-oss-120b": "openai/gpt-oss-120b",
    "gpt-5-mini": "openai/gpt-5-mini",
    "gpt-5-nano": "openai/gpt-5-nano",
    "grok-code-fast1": "x-ai/grok-code-fast-1",
    "grok-4-fast": "x-ai/grok-4-fast",
    "minimax-m2": "minimax/minimax-m2",
}

# Select which model to use
MODEL_KEY = "kimi-k2-thinking"
MODEL_ID = MODELS_OPENROUTER[MODEL_KEY]

model = OpenAIServerModel(
    model_id=MODEL_ID,
    api_base="https://openrouter.ai/api/v1",
    api_key=openrouter_key
)
print(f"✓ Using OpenRouter model: {MODEL_ID}")


✓ Using OpenRouter model: moonshotai/kimi-k2-thinking


## 3.4 Testing Agent

In [116]:
# Use the model instance created in the previous cell
MAX_STEPS = 7

agent = CodeAgent(
    tools=[],
    model=model,  # Use the model instance from previous cell
    additional_authorized_imports=["numpy", "pandas", "json", "csv", "os", "glob", "markdown"],
    max_steps=MAX_STEPS,
    verbosity_level=3,
)

In [117]:
PROMPT = """You are an expert data analyst and you will answer factoid questions by loading and referencing the files/documents listed below.
You have these files available:
{context_files}
Don't forget to reference any documentation in the data dir before answering a question.

IMPORTANT: Use pandas (pd.read_csv) to read CSV files, json.load() for JSON files, and the markdown module to read markdown files. Do NOT use the built-in open() function directly.

Here is the question you need to answer:
{question}

Here are the guidelines you must follow when answering the question above:
{guidelines}
"""
question = "What are the unique set of merchants in the payments data?"
guidelines = "Answer with a comma separated list"

PROMPT = PROMPT.format(
    context_files=CONTEXT_FILENAMES,
    question=question,
    guidelines=guidelines
)

agent_start_time = time.time()
answer = agent.run(PROMPT)
agent_end_time = time.time()

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ You are an expert data analyst and you will answer factoid questions by loading and referencing the             │
│ files/documents listed below.                                                                                   │
│ You have these files available:                                                                                 │
│ ['/home/mykola/repos/dabstep_test/data/context_files/data/context/acquirer_countries.csv',                      │
│ '/home/mykola/repos/dabstep_test/data/context_files/data/context/payments-readme.md',                           │
│ '/home/mykola/repos/dabstep_test/data/context_files/data/context/payments.csv',                                 │
│ '/home/mykola/repos/dabstep_test/data/context_files/data/context/merchant_category_codes.csv',                  │
│ '/home/mykola/repos/dabstep_test/data/context_files/data/context/fees.json',                                    │
│ '/home/mykola/repos/dabstep_test/data/context_files/data/context/merchant_data.json',                           │
│ '/home/mykola/repos/dabstep_test/data/context_files/data/context/manual.md'\]                                   │
│ Don't forget to reference any documentation in the data dir before answering a question.                        │
│                                                                                                                 │
│ IMPORTANT: Use pandas (pd.read_csv) to read CSV files, json.load() for JSON files, and the markdown module to   │
│ read markdown files. Do NOT use the built-in open() function directly.                                          │
│                                                                                                                 │
│ Here is the question you need to answer:                                                                        │
│ What are the unique set of merchants in the payments data?                                                      │
│                                                                                                                 │
│ Here are the guidelines you must follow when answering the question above:                                      │
│ Answer with a comma separated list                                                                              │
│                                                                                                                 │
╰─ OpenAIServerModel - moonshotai/kimi-k2-thinking ───────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Thought: I need to answer the question: "What are the unique set of merchants in the payments data?" I have a list 
of files, including payments.csv, payments-readme.md, manual.md, and other files. To answer, I should:             
1. Read the documentation (payments-readme.md and manual.md) to understand the schema and identify which column in 
the payments data corresponds to merchants.                                                                        
2. Load payments.csv using pandas.                                                                                 
3. Identify the column containing merchant identifiers.                                                            
4. Extract the unique values from that column.                                                                     
5. Return them as a comma-separated list.                                                                          
                                                                                                                   
I'll start by reading the markdown documentation files using the markdown module, then load the CSV, and inspect   
its columns.                                                                                                       
                                                                                                                   
<code>                                                                                                             
import markdown                                                                                                    
import re                                                                                                          
import pandas as pd                                                                                                
                                                                                                                   
# Function to strip HTML tags from markdown output                                                                 
def strip_html(html):                                                                                              
    return re.sub('<[^<]+?>', '', html)                                                                            
                                                                                                                   
# Read payments-readme.md                                                                                          
readme_html =                                                                                                      
markdown.markdownFromFile(input='/home/mykola/repos/dabstep_test/data/context_files/data/context/payments-readme.md
', output=None)                                                                                                    
readme_text = strip_html(readme_html)                                                                              
print("=== payments-readme.md content ===")                                                                        
print(readme_text[:2000])  # print first part to inspect                                                           
                                                                                                                   
# Read manual.md                                                                                                   
manual_html =                                                                                                      
markdown.markdownFromFile(input='/home/mykola/repos/dabstep_test/data/context_files/data/context/manual.md',       
output=None)                                                                                                       
manual_text = strip_html(manual_html)                   

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  import markdown                                                                                                  
  import re                                                                                                        
  import pandas as pd                                                                                              
                                                                                                                   
  # Function to strip HTML tags from markdown output                                                               
  def strip_html(html):                                                                                            
      return re.sub('<[^<]+?>', '', html)                                                                          
                                                                                                                   
  # Read payments-readme.md                                                                                        
  readme_html =                                                                                                    
  markdown.markdownFromFile(input='/home/mykola/repos/dabstep_test/data/context_files/data/context/payments-readm  
  e.md', output=None)                                                                                              
  readme_text = strip_html(readme_html)                                                                            
  print("=== payments-readme.md content ===")                                                                      
  print(readme_text[:2000])  # print first part to inspect                                                         
                                                                                                                   
  # Read manual.md                                                                                                 
  manual_html =                                                                                                    
  markdown.markdownFromFile(input='/home/mykola/repos/dabstep_test/data/context_files/data/context/manual.md',     
  output=None)                                                                                                     
  manual_text = strip_html(manual_html)                                                                            
  print("\n=== manual.md content ===")                                                                             
  print(manual_text[:2000])                                                                                        
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Code execution failed at line 'readme_html = 
markdown.markdownFromFile(input='/home/mykola/repos/dabstep_test/data/context_files/data/context/payments-readme.md
', output=None)' due to: AttributeError: 'OutStream' object has no attribute 'buffer'

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 5 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
 Thought:                                                                                                          
                                                                                                                   
I have successfully identified all unique merchants in the payments data. The `merchant` column in the payments.csv
file contains 5 unique merchant names, which I have extracted, sorted alphabetically, and formatted as a           
comma-separated list.                                                                                              
                                                                                                                   
<code>                                                                                                             
final_answer("Belles_cookbook_store, Crossfit_Hanna, Golfclub_Baron_Friso, Martinis_Fine_Steakhouse, Rafa_AI")     
</code>                                                                                                            
Calling tools:                                                                                                     
[{'id': 'call_5', 'type': 'function', 'function': {'name': 'final_answer', 'arguments': {'answer':                 
'Belles_cookbook_store, Crossfit_Hanna, Golfclub_Baron_Friso, Martinis_Fine_Steakhouse, Rafa_AI'}}}]               
</code>                                                                                                            

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("Belles_cookbook_store, Crossfit_Hanna, Golfclub_Baron_Friso, Martinis_Fine_Steakhouse, Rafa_AI")   
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: Belles_cookbook_store, Crossfit_Hanna, Golfclub_Baron_Friso, Martinis_Fine_Steakhouse, Rafa_AI

[Step 5: Duration 3.67 seconds| Input tokens: 16,891 | Output tokens: 4,250]

In [118]:
notebook_end_time = time.time()

# You can inspect the steps taken by the agent by doing this
def clean_reasoning_trace(trace: list) -> list:
  for step in trace:
      # Remove memory from logs to make them more compact.
      if hasattr(step, "memory"):
          step.memory = None
      if isinstance(step, ActionStep):
          step.agent_memory = None
  return trace

# Access agent's reasoning trace
if hasattr(agent, 'memory') and hasattr(agent.memory, 'steps'):
    steps = agent.memory.steps
    for step in clean_reasoning_trace(steps):
        print(step)
else:
    print("Agent steps not available")

TaskStep(task="You are an expert data analyst and you will answer factoid questions by loading and referencing the files/documents listed below.\nYou have these files available:\n['/home/mykola/repos/dabstep_test/data/context_files/data/context/acquirer_countries.csv', '/home/mykola/repos/dabstep_test/data/context_files/data/context/payments-readme.md', '/home/mykola/repos/dabstep_test/data/context_files/data/context/payments.csv', '/home/mykola/repos/dabstep_test/data/context_files/data/context/merchant_category_codes.csv', '/home/mykola/repos/dabstep_test/data/context_files/data/context/fees.json', '/home/mykola/repos/dabstep_test/data/context_files/data/context/merchant_data.json', '/home/mykola/repos/dabstep_test/data/context_files/data/context/manual.md']\nDon't forget to reference any documentation in the data dir before answering a question.\n\nIMPORTANT: Use pandas (pd.read_csv) to read CSV files, json.load() for JSON files, and the markdown module to read markdown files. Do NO

In [119]:
# Debug: Inspect step object structure
if hasattr(agent, 'memory') and hasattr(agent.memory, 'steps'):
    steps_list = agent.memory.steps
    if len(steps_list) > 0:
        print("First step object details:")
        first_step = steps_list[0]
        print(f"Type: {type(first_step)}")
        print(f"All attributes: {[attr for attr in dir(first_step) if not attr.startswith('_')]}")
        print(f"\nStep object repr:\n{first_step}")


First step object details:
Type: <class 'smolagents.memory.TaskStep'>
All attributes: ['dict', 'task', 'task_images', 'to_messages']

Step object repr:
TaskStep(task="You are an expert data analyst and you will answer factoid questions by loading and referencing the files/documents listed below.\nYou have these files available:\n['/home/mykola/repos/dabstep_test/data/context_files/data/context/acquirer_countries.csv', '/home/mykola/repos/dabstep_test/data/context_files/data/context/payments-readme.md', '/home/mykola/repos/dabstep_test/data/context_files/data/context/payments.csv', '/home/mykola/repos/dabstep_test/data/context_files/data/context/merchant_category_codes.csv', '/home/mykola/repos/dabstep_test/data/context_files/data/context/fees.json', '/home/mykola/repos/dabstep_test/data/context_files/data/context/merchant_data.json', '/home/mykola/repos/dabstep_test/data/context_files/data/context/manual.md']\nDon't forget to reference any documentation in the data dir before answering

In [120]:
# Calculate total tokens from agent execution
from smolagents.memory import ActionStep

total_input_tokens = 0
total_output_tokens = 0

# Extract token information from ActionStep objects in agent memory
if hasattr(agent, 'memory') and hasattr(agent.memory, 'steps'):
    for step in agent.memory.steps:
        if isinstance(step, ActionStep) and hasattr(step, 'token_usage'):
            token_usage = step.token_usage
            if token_usage is not None:
                # token_usage is typically a dict with 'input_tokens' and 'output_tokens'
                if isinstance(token_usage, dict):
                    total_input_tokens += token_usage.get('input_tokens', 0)
                    total_output_tokens += token_usage.get('output_tokens', 0)
                else:
                    # If it's an object with attributes
                    if hasattr(token_usage, 'input_tokens'):
                        total_input_tokens += token_usage.input_tokens
                    if hasattr(token_usage, 'output_tokens'):
                        total_output_tokens += token_usage.output_tokens

print(f"\nTotal Input Tokens: {total_input_tokens}")
print(f"Total Output Tokens: {total_output_tokens}")


Total Input Tokens: 16891
Total Output Tokens: 4250


In [121]:
# Test Summary - Steps, Timing, Answer, and Persistence
import os
import csv
from datetime import datetime
import glob

print("\n" + "="*80)
print("TEST SUMMARY")
print("="*80)

# Print model and question info
print(f"\n📊 Model: {MODEL_ID}")
print(f"❓ Question: {question}")

# Calculate metrics
total_steps = 0
total_time = 0
if hasattr(agent, 'memory') and hasattr(agent.memory, 'steps'):
    steps_list = agent.memory.steps
    total_steps = len(steps_list)
    print(f"\n📈 Total Steps: {total_steps}")
    
    # Calculate agent execution time
    try:
        total_time = agent_end_time - agent_start_time
        print(f"\n⏱️  Total Agent Time: {total_time:.2f}s")
        print(f"   Notebook Runtime: {notebook_end_time - notebook_start_time:.2f}s")
    except NameError:
        print("\n⚠️  Agent timing information not available (run the agent execution cell first)")

# Print final answer
print(f"\n💬 Final Answer:")
print(f"{answer}")

# Print token info
print(f"\n🔤 Tokens:")
print(f"   Input Tokens: {total_input_tokens}")
print(f"   Output Tokens: {total_output_tokens}")

# Prepare results directory - save to repo/results/toy_results with timestamped filename
# Navigate from notebooks directory to repo root
repo_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
results_dir = os.path.join(repo_root, "results", "toy_results")
os.makedirs(results_dir, exist_ok=True)

# Use timestamped filename to keep old results separate
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
results_file = os.path.join(results_dir, f"results_with_open_fix_{timestamp}.csv")

# Prepare result row
result_row = {
    "timestamp": datetime.now().isoformat(),
    "model": MODEL_ID,
    "question": question,
    "total_steps": total_steps,
    "total_agent_time_s": round(total_time, 2),
    "notebook_runtime_s": round(notebook_end_time - notebook_start_time, 2),
    "input_tokens": total_input_tokens,
    "output_tokens": total_output_tokens,
    "answer": str(answer)
}

# Save to CSV
file_exists = os.path.isfile(results_file)
with open(results_file, 'a', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=result_row.keys())
    if not file_exists:
        writer.writeheader()
    writer.writerow(result_row)

print(f"\n✅ Results saved to: {results_file}")

# Display all results from ALL runs (combine all timestamped files)
print("\n" + "="*80)
print("ALL RESULTS (ALL RUNS)")
print("="*80)

import pandas as pd

# Find all results files matching the pattern
all_results_files = sorted(glob.glob(os.path.join(results_dir, "results_with_open_fix_*.csv")))

if all_results_files:
    # Read and combine all results files
    all_results = []
    for results_file_path in all_results_files:
        df = pd.read_csv(results_file_path)
        all_results.append(df)
    
    results_df = pd.concat(all_results, ignore_index=True)
    print("\n")
    print(results_df.to_string(index=False))
else:
    print("No results files found")
    results_df = pd.DataFrame()

print("\n" + "="*80)


TEST SUMMARY

📊 Model: moonshotai/kimi-k2-thinking
❓ Question: What are the unique set of merchants in the payments data?

📈 Total Steps: 6

⏱️  Total Agent Time: 71.77s
   Notebook Runtime: 72.72s

💬 Final Answer:
Belles_cookbook_store, Crossfit_Hanna, Golfclub_Baron_Friso, Martinis_Fine_Steakhouse, Rafa_AI

🔤 Tokens:
   Input Tokens: 16891
   Output Tokens: 4250

✅ Results saved to: /home/mykola/repos/dabstep_test/results/toy_results/results_with_open_fix_20251111_172335.csv

ALL RESULTS (ALL RUNS)


                 timestamp                       model                                                   question  total_steps  total_agent_time_s  notebook_runtime_s  input_tokens  output_tokens                                                                                         answer
2025-11-11T17:06:04.551948            x-ai/grok-4-fast What are the unique set of merchants in the payments data?            4               14.74               15.95          8942           2707     

In [122]:
results_df

,timestamp,model,question,total_steps,total_agent_time_s,notebook_runtime_s,input_tokens,output_tokens,answer
0,2025-11-11T17:06:04.551948,x-ai/grok-4-fast,What are the unique set of merchants in the pa...,4,14.74,15.95,8942,2707,"Belles_cookbook_store,Crossfit_Hanna,Golfclub_..."
1,2025-11-11T17:06:29.566669,x-ai/grok-4-fast,What are the unique set of merchants in the pa...,4,9.41,10.65,8517,1688,"Crossfit_Hanna,Belles_cookbook_store,Golfclub_..."
2,2025-11-11T17:08:38.253162,x-ai/grok-4-fast,What are the unique set of merchants in the pa...,6,15.69,-6.02,16394,2659,"Belles_cookbook_store, Crossfit_Hanna, Golfclu..."
3,2025-11-11T17:09:39.490712,x-ai/grok-4-fast,What are the unique set of merchants in the pa...,2,6.37,-12.64,2360,942,"Belles_cookbook_store,Crossfit_Hanna,Golfclub_..."
4,2025-11-11T17:12:38.314130,x-ai/grok-4-fast,What are the unique set of merchants in the pa...,4,8.78,10.13,9187,1304,"Belles_cookbook_store, Crossfit_Hanna, Golfclu..."
5,2025-11-11T17:13:35.201777,qwen/qwen3-coder,What are the unique set of merchants in the pa...,3,10.44,11.41,4891,128,"Crossfit_Hanna, Belles_cookbook_store, Golfclu..."
6,2025-11-11T17:15:07.367432,qwen/qwen3-coder,What are the unique set of merchants in the pa...,3,12.35,13.43,4889,124,"Crossfit_Hanna, Belles_cookbook_store, Golfclu..."
7,2025-11-11T17:16:58.018352,openai/gpt-5-mini,What are the unique set of merchants in the pa...,3,56.10,57.19,6032,3527,"Belles_cookbook_store, Crossfit_Hanna, Golfclu..."
8,2025-11-11T17:18:05.009106,openai/gpt-oss-120b,What are the unique set of merchants in the pa...,4,10.83,11.90,7404,931,"Belles_cookbook_store, Crossfit_Hanna, Golfclu..."
9,2025-11-11T17:19:08.738898,z-ai/glm-4.5-air,What are the unique set of merchants in the pa...,3,11.45,12.54,5602,357,"Belles_cookbook_store, Crossfit_Hanna, Golfclu..."


In [123]:
print(f"Notebook runtime: {notebook_end_time - notebook_start_time:.2f} seconds")
print(f'Right now is {datetime.now().isoformat()}')

Notebook runtime: 72.72 seconds
Right now is 2025-11-11T17:23:35.675220
